In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab4.ipynb")

<img src="img/dsci511_header.png" width="600">

# Lab 4: Advanced Wrangling with Pandas

## Instructions

rubric={mechanics:4}

Follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
plt.rcParams.update({'font.size': 14, 'axes.labelweight': 'bold'})

## Exercise 1

rubric={autograde:4}

In Lab 3, you performed some basic Pandas operations on the Gapminder dataset. However, this dataset was given to you clean and shiny and ready-to-go. In the real world, that's rarely the case, and in this exercise you'll have to clean up a "dirty" version of the Gapminder dataset.

Your goal is to load in "dirty Gapminder" as a dataframe called `dirty` and "clean Gapminder" as a dataframe called `clean`, and wrangle `dirty` until it is the same as `clean`:
- Dirty Gapminder: <https://raw.githubusercontent.com/STAT545-UBC/STAT545-UBC.github.io/master/gapminderDataFiveYear_dirty.txt>
- Clean Gapminder: <https://raw.githubusercontent.com/STAT545-UBC/STAT545-UBC.github.io/master/gapminderDataFiveYear.txt>

A test has been provided to check that `dirty` is the same as `clean`. Things you might want to do to clean up `dirty`:

- Check that `dirty` and `clean` have the same columns;
- Check if there is any missing data, if there is missing data (NaNs or empty strings) fill them with sensible values;
- Check for things like capitalization, spelling, etc;
- There may be entries that appear to have the exact same spelling and capitalization in both `dirty` and `clean`, but still don't match... Extra whitespace is often a frustrating (and invisible) problem when wrangling text data. You can use `Series.str.strip()` to trim any additional unwanted whitespace around a string.
- At any time, you can check which rows in `dirty` are not equal to `clean` using something like: `dirty[dirty.ne(clean).any(axis=1)]`.

In [ ]:
...

In [ ]:
grader.check("q1")

## Exercise 2

rubric={autograde:4}

Suppose that you are hired as a data scientist for the [Narrabeen](http://narrabeen.wrl.unsw.edu.au/background/#overview) beach survey program in Sydney, Australia. The survey program started in the 1970's and has continued to the present day. All the data is available at: <http://narrabeen.wrl.unsw.edu.au/explore_data/time_series/>.

In the next few exercises we're going to download, wrangle, and explore this time-series dataset.

<img src="img/narrabeen.jpg" width="400">

The survey program is aimed to measure the width of the beach every few weeks. There are five locations along the beach for which measurements are made, from location 1 at the northern end of the beach, to location 5 at the southern end:

![](img/survey_locations.jpg)

A function `get_beach_data()` is provided for you in the `scraper.py` module that "scrapes" the <http://narrabeen.wrl.unsw.edu.au> website for data using the `requests` library (you'll learn more about web scraping in DSCI 525).
This function returns a list of tuples containing the date of a survey and the width of the beach on that date, for the given `survey_location` (i.e., 1, 2, 3, 4, or 5). For example:

```python
>>>get_beach_data(survey_location=1)
[('1976-04-27', '78.5'),
 ('1976-05-10', '65.1'),
 ('1976-05-18', '72.9'),
 ('1976-05-25', '76.0'),
 ('1976-06-02', '83.4'),
 ('1976-06-16', '67.7'),
 ('1976-06-18', '74.5'),
 ('1976-06-23', '78.2'),
 .
 .
 .
```

**Your Tasks:**

1. Import the `get_beach_data()` function from the `scraper.py` module.

1. Extract the data for each survey location (1, 2, 3, 4, 5) and store them in five separate `Series` with the Index of the `Series` being the datetime. You'll need to convert the string dates to datetimes with `pd.to_datetime()`.

1. Merge all of the `Series` together in a single dataframe. Note that the `Series` data are not all the same length (some measurements are missing), so we want to do an "inner" join on the index values between all the `Series` here.

Your final dataframe should be called `beach_df` and should look like this (note that the datetimes have been set as the index):

```
            location_1  location_2  location_3  location_4  location_5
1976-04-27        78.5        59.7        44.7        16.2        63.5
1976-05-10        65.1        60.7        61.6        15.0        65.8
1976-05-18        72.9        67.0        68.6        23.8        64.3
1976-05-25        76.0        70.3        67.6        21.8        44.5
1976-06-02        83.4        79.5        79.8        24.5        57.1
...                ...         ...         ...         ...         ...
```

The next few questions of this lab rely on this dataframe, so make sure that you pass the autograder tests before proceeding.

If you are unable to pass the tests, I've saved a version of the dataframe in the data folder of this directory. When you get to Exercise 3, you may just read that in with

```python
pd.read_csv('data/beach_data.csv', index_col=0, parse_dates=True)
```

In [ ]:
...

In [ ]:
grader.check("q2")

## Exercise 3

rubric={accuracy:4,efficiency:2}

In Exercise 2, you may have noticed that the frequency of the data you just collected is on the order of a few days to a few weeks—it's irregularly spaced.
This can make it difficult to extract patterns from the data. One way to deal with this is to resample the data to regular intervals (e.g. from irregular daily data to monthly data).

**Note:** If you were unable to wrangle the data and pass the autograder tests in Exercise 2, you can find a version of the `beach_df` in the `data` folder which you can read in using:

```python
pd.read_csv('data/beach_data.csv', index_col=0, parse_dates=True)
```

**Your Tasks:**

1. First, subtract the mean value of each column from the same column. This will help to see if a location on the beach is narrower (negative numbers) or wider (positive numbers) at a certain time compared to the average.

1. Resample `beach_df` from the previous question to monthly intervals with the mean as the aggregation function. Use `.resample()` in a way that it converts the index type of the dataframe to `PeriodIndex`.

3. Remove all entries prior to 1980. We do this because the accuracy of these measurements is not great.
**Hint:** Remember that you can use datetime indexing in Pandas with [partial string matching](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#partial-string-indexing).

1. Finally, melt the dataframe such that your final `beach_df` looks similar to this, and show it in the output:

```
            location      width
datetime                       
1980-01   location_1   4.356982
1980-02   location_1   3.856982
1980-03   location_1   0.156982
1980-04   location_1   4.856982
1980-05   location_1 -14.343018
...              ...        ...
2019-07   location_5  -7.042108
2019-08   location_5  -6.225441
2019-09   location_5 -14.425441
2019-10   location_5 -15.125441
2019-11   location_5 -15.542108
```

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

## Exercise 4

For this exercise, you should use the `beach_df` dataframe resulting from your work in Exercise 3.

### 4.1
rubric={accuracy:1}

By chaining various Pandas methods together and showing the output, answer the following question:

Which survey location has the greatest variability in measured beach widths? Measure variability in terms of standard deviation.

**Note:** Don't overwrite `beach_df`.

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

### 4.2
rubric={accuracy:1}

Filter the dataset in an appropriate way, and show the output to answer the following question:

Which survey location experienced the greatest erosion (i.e. the largest decrease in width) in the entire dataset?

**Note:** Don't overwrite `beach_df`.

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

### 4.3
rubric={accuracy:1}

Write code that shows in the output the top three months during which the beach is narrowest based on the average value for width.

You can leave the output months as numbers inside a dataframe along with the corresponding `width`s.

**Hint:** Decompose the datetime index to extract the month, and use it to create a new `month` column in the original `beach_df` dataframe. After this, you're going to be able to use `.groupby()` to answer this question.

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

### 4.4
rubric={accuracy:1}

Calculate the correlation between beach width observations at location 1 and location 5.

> **Read if you're interested to know the significance of this computation**: One interesting thing about embayed beaches like Narrabeen is that the amount of sand in the system is typically conserved. That means that if one part of the beach is getting narrower, another section is probably getting wider. This is most obvious at the extremes of the beach, location 1 and location 5 in our case. We can confirm this by calculating the correlation between observations of these two locations.

**Note:**

- The correlation value should be negative, which means that if one location is eroding, the other location is doing the opposite.
- There is a dataframe method that calculates correlation between columns which you can use.
- Don't overwrite `beach_df`.

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

### 4.5
rubric={accuracy:1}

Perform a double `.groupby()` to determine the **month** AND **location** for which the beach is the widest.

**Note:** Whenever you do a `.groupby()` operation, you need an aggregation function to summarize information in multiple rows in just one row (we'll learn more about these kinds of operations in DSCI 513).
Here, use the median aggregator function to summarize information in rows with similar month and location values, and then try to find what month-location pair that corresponds to the maximum measurement for beach width.

> **Read if you're interested to know the significance of this computation**: In the previous questions we learned that:
>
>- The northern-most beach location (location 1) is the most exposed to waves,
>- The biggest waves occur in Australia's winter, but that if one part of the beach is becoming narrower, then the opposite end is probably getting wider.
>
>Therefore we'd expect the answer here to be location 5 (opposite end of the beach to location 1) some time around the middle of the year (winter-ish time in Australia).

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

### 4.6
rubric={accuracy:1}

For how many months in our monthly resampled date spanning 1980-2019 do we have at least one missing observation for a location?

**Hint:** Use the `.any()` method of Pandas dataframes.

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

## Exercise 5

rubric={accuracy:1}

In Exercise 4, we learned that there is a "rotation effect" at Narrabeen Beach where if one end of the beach is getting narrower, often the other end is getting wider. This is most obvious at the extremes of the beach; that is, location 1 and location 5.

Your task here is to show a visual confirmation of the rotation phenomenon by plotting the observations for location 1 and location 5 on the same figure. Remember that Pandas has a built-in plotting method for its dataframes.

- Resample the data to your desired periods. The choice of resampling period should be made such that it helps you better see the patterns.

- Drop `NaN` values before plotting.

<!-- BEGIN QUESTION -->



In [ ]:
...

<!-- END QUESTION -->

